# Microblaze Python Bindings

In addition to the `%%microblaze` magic there is also a set of Python bindings for the core API. These bindings are designed to provide an easy to use way of making use of the APIs with the full functionality of Python available. As the API is as close to identical as possible to the C API, any performance critical code can be moved to C with minimal effort.

The two classes are `MbioBinding` which provides just the core `mbio.h` functions and `IopSwitchBinding` which provides the additional functionality from the `iop_switch.h` functions for use with the PMOD and Arduino IOPs.

The constructors for both take the IOP to run on and optionally an array of modules that can provide additional functionality. For this example we are going to concentrate on the `IopSwitchBinding` as we are using the Base overlay PMOD and Arduino IOPs.

First step is to load the Base Overlay and create an instance of `IopSwitchBinding` on PMODA

In [1]:
from pynq.overlays.base import BaseOverlay
from ipython_microblaze import IopSwitchBinding

base = BaseOverlay('base.bit')
iop = IopSwitchBinding(base.PMODA)

We can now inspect the `iop` variable to find all of the functions that have been wrapped.

In [2]:
dir(iop)

['G1',
 'G2',
 'G3',
 'G4',
 'GPIO_IN',
 'GPIO_OUT',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_build_constants',
 '_build_functions',
 '_mb',
 '_populate_typedefs',
 'delay',
 'delayMicroseconds',
 'iop_switch_analog_grove',
 'iop_switch_analog_raw',
 'iop_switch_gpio_grove',
 'iop_switch_gpio_raw',
 'iop_switch_i2c_grove',
 'iop_switch_i2c_raw',
 'iop_switch_init',
 'iop_switch_pwm_grove',
 'iop_switch_pwm_raw',
 'mb_adc_num_devices',
 'mb_adc_open',
 'mb_adc_range',
 'mb_adc_read',
 'mb_adc_read_raw',
 'mb_adc_vref',
 'mb_gpio_num_devices',
 'mb_gpio_open_all',
 'mb_gpio_open_pin',
 'mb_gpio_open_range',
 'mb_gpio_read',
 'mb_gpio_set_direction',
 'mb_gpio_wr

You can see that all of the functions from the other notebooks are here along with the enum constants. We can now use these functions to turn on an LED attached to G2

In [3]:
iop.iop_switch_init()
led = iop.iop_switch_gpio_grove(iop.G2, 0)
iop.mb_gpio_set_direction(led, iop.GPIO_OUT)
iop.mb_gpio_write(led, 1)

To make the API slightly easier to use in python, each `typedef` in the original source is wrapped as a custom python class. Any methods with that type name as prefix are then added as member functions. Using this API we can write a flashing script in a more python-like way

In [4]:
import time
for _ in range(10):
    led.write(0)
    time.sleep(0.2)
    led.write(1)
    time.sleep(0.2)

A more complex API is that of the I2C controller which has to send and receive variable strings of bytes. In Python these are `bytes` objects for const strings or `bytearrays` for non-const strings. Other array types of size 1-byte may also work but haven't been thoroughly tested yet. Copying the example from a previous notebook, let's read from an I2C attached ADC. The original C code looked like:

```C
#define ADDRESS 0x50

i2c bus = i2c_connect_grove(G4);
unsigned char data[2];
// Setup the device
data[0] = 0x02;
data[1] = 0x20;
i2c_write(bus, ADDRESS, data, 2);
// Read the value
data[0] = 0x00;
i2c_write(bus, ADDRESS, data, 1);
i2c_read(bus, ADDRESS, data, 2);
```

In [5]:
ADDRESS = 0x50

bus = iop.iop_switch_i2c_grove(iop.G4)
data = bytearray(b'\x02\x20')
bus.write(ADDRESS, data, 2)
data[0] = 0x00
bus.write(ADDRESS, data, 1)
bus.read(ADDRESS, data, 2)

data

bytearray(b'\x00N')

More complex peripherals are exposed in C as modules. Modules can be added into the set of bindings by passing the names as an additional array argument. For this example we are going to use the Grove LED bar library as it comes with the ipython_microblaze package. Before changing the bindings we need to reset the microblaze.

In [6]:
iop.reset()
iop = IopSwitchBinding(base.PMODA, ['ledbar'])

If we now look at the members of the `iop` variable again we will see that the `ledbar_*` functions now appear.

In [7]:
dir(iop)

['G1',
 'G2',
 'G3',
 'G4',
 'GPIO_IN',
 'GPIO_OUT',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_build_constants',
 '_build_functions',
 '_mb',
 '_populate_typedefs',
 'delay',
 'delayMicroseconds',
 'iop_switch_analog_grove',
 'iop_switch_analog_raw',
 'iop_switch_gpio_grove',
 'iop_switch_gpio_raw',
 'iop_switch_i2c_grove',
 'iop_switch_i2c_raw',
 'iop_switch_init',
 'iop_switch_pwm_grove',
 'iop_switch_pwm_raw',
 'ledbar_open',
 'ledbar_open_grove',
 'ledbar_set_data',
 'ledbar_set_level',
 'mb_adc_num_devices',
 'mb_adc_open',
 'mb_adc_range',
 'mb_adc_read',
 'mb_adc_read_raw',
 'mb_adc_vref',
 'mb_gpio_num_devices',
 'mb_gpio_open_all',
 'mb_gpio_open_pin',

We can now use this API to make an increasing pattern appear on the bar. Same as for the `mb_*` typedefs, the `ledbar_` functions all appear as members of an `ledbar` class.

In [8]:
iop.iop_switch_init()
bar = iop.ledbar_open_grove(iop.G1)
for i in range(11):
    bar.set_level(i)
    time.sleep(0.5)

The bindings are generated dynamically from the C sources and it possible to use other files as the input. The `MicroblazeRPC` class exposes this functionality for general sources. The current restrictions on functions used are:

 * No structs or unions in the interface
 * Only const and non-const char* pointer arguments
 * No pointer return types
 * Typedefs for classes must resolve to basic types

Subject to these restrictions, any code can be run in this environment for experimentation and debugging.